In [ ]:
import numpy as np
from pathlib import Path
from os import urandom as get_random_bytes
from PIL import Image
import cv2

In [ ]:
# PRESENT Algorithm Implementation
Sbox = [0xc, 0x5, 0x6, 0xb, 0x9, 0x0, 0xa, 0xd, 0x3, 0xe, 0xf, 0x8, 0x4, 0x7, 0x1, 0x2]
Sbox_inv = [Sbox.index(x) for x in range(16)]
PBox = [0, 16, 32, 48, 1, 17, 33, 49, 2, 18, 34, 50, 3, 19, 35, 51,
        4, 20, 36, 52, 5, 21, 37, 53, 6, 22, 38, 54, 7, 23, 39, 55,
        8, 24, 40, 56, 9, 25, 41, 57, 10, 26, 42, 58, 11, 27, 43, 59,
        12, 28, 44, 60, 13, 29, 45, 61, 14, 30, 46, 62, 15, 31, 47, 63]
PBox_inv = [PBox.index(x) for x in range(64)]

def generateRoundkeys80(key, rounds):
    roundkeys = []
    for i in range(1, rounds + 1):
        roundkeys.append(key >> 16)
        key = ((key & (2 ** 19 - 1)) << 61) + (key >> 19)
        key = (Sbox[key >> 76] << 76) + (key & (2 ** 76 - 1))
        key ^= i << 15
    return roundkeys

def generateRoundkeys128(key, rounds):
    roundkeys = []
    for i in range(1, rounds + 1):
        roundkeys.append(key >> 64)
        key = ((key & (2 ** 67 - 1)) << 61) + (key >> 67)
        key = (Sbox[key >> 124] << 124) + (Sbox[(key >> 120) & 0xF] << 120) + (key & (2 ** 120 - 1))
        key ^= i << 62
    return roundkeys

def addRoundKey(state, roundkey):
    return state ^ roundkey

def sBoxLayer(state):
    output = 0
    for i in range(16):
        output += Sbox[(state >> (i * 4)) & 0xF] << (i * 4)
    return output

def sBoxLayer_dec(state):
    output = 0
    for i in range(16):
        output += Sbox_inv[(state >> (i * 4)) & 0xF] << (i * 4)
    return output

def pLayer(state):
    output = 0
    for i in range(64):
        output += ((state >> i) & 0x01) << PBox[i]
    return output

def pLayer_dec(state):
    output = 0
    for i in range(64):
        output += ((state >> i) & 0x01) << PBox_inv[i]
    return output

def string2number(i):
    if isinstance(i, bytes):
        return int.from_bytes(i, 'big')
    else:
        b = i.encode('utf-8')
        return int.from_bytes(b, 'big')

def number2string_N(i, N):
    return i.to_bytes(N, 'big')


class Present:
    def __init__(self, key, rounds=32):
        self.rounds = rounds
        if len(key) * 8 == 80:
            self.roundkeys = generateRoundkeys80(string2number(key), self.rounds)
        else:
            self.roundkeys = generateRoundkeys128(string2number(key), self.rounds)

    def encrypt(self, block):
        num_blocks = (len(block) + 7) // 8
        encrypted_blocks = []
        for i in range(num_blocks):
            start = i * 8
            end = min((i + 1) * 8, len(block))
            block_data = block[start:end].ljust(8, b'\x00')
            encrypted_blocks.append(self.encrypt_block(block_data))
        return b''.join(encrypted_blocks)

    def decrypt(self, block):
        num_blocks = (len(block) + 7) // 8
        decrypted_blocks = []
        for i in range(num_blocks):
            start = i * 8
            end = min((i + 1) * 8, len(block))
            block_data = block[start:end].ljust(8, b'\x00')
            decrypted_blocks.append(self.decrypt_block(block_data))
        return b''.join(decrypted_blocks)

    def encrypt_block(self, block):
        state = string2number(block)
        for i in range(self.rounds - 1):
            state = addRoundKey(state, self.roundkeys[i])
            state = sBoxLayer(state)
            state = pLayer(state)
        cipher = addRoundKey(state, self.roundkeys[-1])
        return number2string_N(cipher, 8)

    def decrypt_block(self, block):
        state = string2number(block)
        for i in range(self.rounds - 1):
            state = addRoundKey(state, self.roundkeys[-i - 1])
            state = pLayer_dec(state)
            state = sBoxLayer_dec(state)
        decipher = addRoundKey(state, self.roundkeys[0])
        return number2string_N(decipher, 8)

In [ ]:
# Unified function to calculate metrics
def calculate_metrics(original_image, encrypted_image):
    if original_image is None or encrypted_image is None:
        print("Error: Unable to load image(s).")
        return

    # NPCR
    num_changed_pixels = np.sum(original_image != encrypted_image)
    total_pixels = original_image.size
    npcr = (num_changed_pixels / total_pixels) * 100

    # UACI
    abs_diff = np.abs(original_image - encrypted_image)
    uaci = np.mean(abs_diff / 255) * 100

    # Mean Deviation
    mean_deviation = np.mean(np.abs(original_image - encrypted_image))

    # MSE
    mse = np.mean((original_image - encrypted_image) ** 2)

    # PSNR
    max_intensity = 255
    psnr = 10 * np.log10(max_intensity ** 2 / mse)

    # Entropy
    flattened = encrypted_image.flatten()
    histogram = np.histogram(flattened, bins=256, range=(0, 255))[0]
    probabilities = histogram / float(len(flattened))
    probabilities = probabilities[probabilities > 0]
    entropy = -np.sum(probabilities * np.log2(probabilities))

    return npcr, uaci, mean_deviation, mse, psnr, entropy

In [ ]:
Path("PRESENT Encryption Files").mkdir(parents=True, exist_ok=True)

img_path = r"Enter the path of the input image"
image = Image.open(img_path).resize((256, 256)).convert("L")
original_image_path = "PRESENT Encryption Files/original_image.png"
encrypted_image_path = "PRESENT Encryption Files/encrypted_image.png"
image.save(original_image_path)

# Convert image to bytes
image_np = np.array(image)
image_bytes = image_np.tobytes()

# Key generation
key = get_random_bytes(16)
cipher = Present(key)

# Encrypt and decrypt
encrypted = cipher.encrypt(image_bytes)
decrypted = cipher.decrypt(encrypted)

# Convert encrypted data back to image
hex_string = encrypted.hex()
encrypted_array = np.array([int(hex_string[i:i+2], 16) for i in range(0, len(hex_string), 2)], dtype=np.uint8)
encrypted_array = encrypted_array[:image_np.size].reshape(image_np.shape)
encrypted_image = Image.fromarray(encrypted_array)
encrypted_image.save(encrypted_image_path)

In [ ]:
# Load images for metric computation
original_image = cv2.imread(original_image_path, cv2.IMREAD_GRAYSCALE)
encrypted_image = cv2.imread(encrypted_image_path, cv2.IMREAD_GRAYSCALE)

# Compute metrics
npcr, uaci, mean_deviation, mse, psnr, entropy = calculate_metrics(original_image, encrypted_image)

# Display metrics
print("NPCR score:", npcr)
print("UACI score:", uaci)
print("Mean Deviation:", mean_deviation)
print("Mean Squared Error (MSE):", mse)
print("Peak Signal-to-Noise Ratio (PSNR):", psnr)
print("Entropy of the encrypted image data:", entropy)